In [25]:
import pandas as pd
# from scipy.stats import pearsonr

#create DataFrame
df = pd.read_csv('data/2022.csv')
highest = {}

cols = df.columns
cols = cols.drop('date').drop('bottles drank').drop('times peed').drop('time slept').drop('time awaked').drop('total sleep time').drop('effectiveness (Am I SPENDING the RIGHT TIME)').drop('Sleep Min').drop('Sleep Hr').drop('Awake Hr').drop('% of planned output').drop("Time slept").drop("Time woken")

index =4
# calculate correlation coefficient between all cols with overall score
for j in range(0, len(cols)):
    if j == index or j == len(cols) + index:
        continue
    col1 = cols[index]
    col2 = cols[j]
    # try:
    # corr, p = pearsonr(df[col1].dropna(), df[col2].dropna())
    # except:
    corr = df[col1].corr(df[col2])
    # p=None
    highest[col2] = f'{corr}'#, {p}'

highest = sorted(highest.items(), key=lambda x: x[1], reverse=True)
print(f'Correlation coefficient between {cols[index]} and other columns: \n')
for k, v in highest:
    print(k, v, sep=':\t')

Correlation coefficient between energy and other columns: 

health:	0.5967354745951695
excitement:	0.5551438239214108
overall:	0.5261207810259757
happiness:	0.5122837330771041
positivity:	0.49115055863837925
purpose:	0.4430439837013459
moment:	0.42297954122249526
sociability:	0.41106973051669865
sleep score:	0.2442105123534386
Minutes Asleep:	0.2351638849373829
integrity:	0.22049047112746473
Time in Bed:	0.2153057487148139
Minutes REM Sleep:	0.1224890481480565
Minutes Deep Sleep:	0.08996315680582072
sleep duration score:	0.0868760544803273
Distance:	0.08340820155849758
Steps:	0.08047801037051411
Calories Burned:	0.058589522251430895
Floors:	0.05386557976530006
stress score:	0.049712848247205454
Minutes Very Active:	0.030690360387101018
Minutes Light Sleep:	0.029141931219186193
Activity Calories:	0.020875112392800762
Minutes Awake:	0.01723274430478346
scrobbles:	0.01578276630189171
Minutes Fairly Active:	0.011214298532673103
Number of Awakenings:	0.00905977010540489
Minutes Lightly Acti

In [6]:
import pandas as pd


#create DataFrame
df = pd.read_csv('data/2022.csv')
highest = {}

cols = df.columns
cols = cols.drop('date').drop('bottles drank').drop('times peed').drop('time slept').drop('time awaked').drop('total sleep time').drop('effectiveness (Am I SPENDING the RIGHT TIME)').drop('Sleep Min').drop('Sleep Hr').drop('Awake Hr').drop('% of planned output').drop("Time slept").drop("Time woken")


# calculate correlation coefficient between all cols
for i in range(len(cols)):
    for j in range(i+1, len(cols)):
        col1 = cols[i]
        col2 = cols[j]
        # try:
        # corr, p = pearsonr(df[col1].dropna(), df[col2].dropna())
        # except:
        corr = df[col1].corr(df[col2])
        # p=None
        highest[col1 + '-' + col2] = f'{corr}'#, {p}'

highest = sorted(highest.items(), key=lambda x: x[1], reverse=True)
for k, v in highest:
    if not (k == 'Steps-Distance' or k == 'Minutes Asleep-Time in Bed'):
        print(k, v, sep=':\t')

overall-happiness:	0.8951773030444721
Time in Bed-Minutes Light Sleep:	0.8558363546778726
Minutes Lightly Active-Activity Calories:	0.8483751340840137
Minutes Asleep-Minutes Light Sleep:	0.8481350107348729
Calories Burned-Steps:	0.8453448495529637
Calories Burned-Distance:	0.8431164527873529
Calories Burned-Activity Calories:	0.8348884367784775
Minutes Asleep-sleep duration score:	0.8285455023220815
Minutes Fairly Active-Activity Calories:	0.7987801652862327
sleep score-sleep duration score:	0.7972886351351555
Minutes Very Active-Activity Calories:	0.7972618080217573
Time in Bed-sleep duration score:	0.7960087322515169
Calories Burned-Minutes Fairly Active:	0.7577102347501551
Steps-Minutes Very Active:	0.7491432238417206
Minutes Awake-Number of Awakenings:	0.748998658278784
Minutes REM Sleep-sleep score:	0.7483706397734553
Distance-Minutes Very Active:	0.7419236264551471
Calories Burned-Minutes Very Active:	0.7328333469284078
Steps-Activity Calories:	0.7305000784959909
happiness-excite

In [ ]:
import pandas as pd
import glob
import os

path = r'/Users/rami/Library/CloudStorage/OneDrive-UniversityofCalgary/code/python/hello_ds/journal-data-analysis/2022-activity' # use your path

all_files = glob.glob(os.path.join(path, "*.csv"))
df = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)

# save to csv
df.to_csv('2022-a.csv', index=False, encoding='utf-8')

path = r'/Users/rami/Library/CloudStorage/OneDrive-UniversityofCalgary/code/python/hello_ds/journal-data-analysis/2022-sleep' # use your path

all_files = glob.glob(os.path.join(path, "*.csv"))
df = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)

# save to csv
df.to_csv('2022-s.csv', index=False, encoding='utf-8')

In [ ]:
# DANGEROUS, remove first line of all files in 'all_files'
for f in all_files:
    with open(f, 'r') as fin:
        data = fin.read().splitlines(True)
    with open(f, 'w') as fout:
        fout.writelines(data[1:])

In [ ]:
import pandas as pd

# Read in the two CSV files
df1 = pd.read_csv("2022-s.csv")
df2 = pd.read_csv("2022-a.csv")

# Merge the two dataframes on a common column
merged_df = df2.join(df1.set_index('Date'), on='Date')

# Save the merged dataframe to a new CSV file
merged_df.to_csv("2022-fb.csv", index=False)

In [9]:
with open('2022-m.csv', 'r') as fin:
    data = fin.read().splitlines(True)
    # get the number of times each date was mentioned
    dates = {}
    for date in data:
        date = date.strip()
        if date in dates:
            dates[date] += 1
        else:
            dates[date] = 1

In [5]:

import pandas as pd

# Read in the two CSV files
df1 = pd.read_csv("2022-fb.csv")
df2 = pd.read_csv("2022-jd.csv")

# Merge the two dataframes on a common column
merged_df = df2.join(df1.set_index('Date'), on='Date')

# add the number of times each date was mentioned to its row
merged_df['scrobbles'] = merged_df['Date'].map(dates)

# Save the merged dataframe to a new CSV file
merged_df.to_csv("2022.csv", index=False)

In [ ]:
import pandas as pd

# Read in the two CSV files
df1 = pd.read_csv("2022.csv")
df2 = pd.read_csv("sleepStress.csv")

# Merge the two dataframes on a common column
merged_df = df1.join(df2.set_index('Date'), on='Date')


# Save the merged dataframe to a new CSV file
merged_df.to_csv("2022.csv", index=False)